In [29]:
import json
import pandas as pd
from src.umls import Umls
umls = Umls('umls/processed')
mrconso = pd.read_feather('umls/processed/mrconso.feather')

In [43]:
def get_name(cui):
    try:
        return mrconso[(mrconso.CUI==cui)&(mrconso.ISPREF=='Y')].values[0,1]
    except:
        print("Failed getting",cui)
        return ""
get_name('C0000039')

'Dipalmitoylphosphatidylcholine (substance)'

In [50]:
with open('results/bb_binary_similarity_nll.json') as f:
    result = json.load(f)['result']

results = []
for r in result:
    mention = r['mention']
    gold_cui = r['golden_cui']
    candidates = r['candidates']

    try:
        first_correct = [c['label'] for c in candidates].index(1)
    except:
        first_correct = -1
    pred = candidates[0]['name']
    pred_dist = umls.dist(gold_cui,candidates[0]['cui'])
    correct = get_name(gold_cui)
    results.append((mention, first_correct, pred_dist, correct, pred))
df = pd.DataFrame(results, columns=['mention','first_correct','pred_dist','correct','pred'])

Failed getting C0178638
Failed getting C1971825
Failed getting C0178638
Failed getting C0162712
Failed getting C0005001
Failed getting C0085934
Failed getting C0064238
Failed getting C0699055
Failed getting C0162712
Failed getting C1456822
Failed getting C0699194
Failed getting C0182440
Failed getting C0591585
Failed getting C0733815
Failed getting C0064956
Failed getting C0728762
Failed getting C0592168
Failed getting C0699187
Failed getting C0086787
Failed getting C0067044
Failed getting C0728763
Failed getting C0086787
Failed getting C0719509
Failed getting C0699129
Failed getting C0593507
Failed getting C0086787
Failed getting C0728763
Failed getting C0086787
Failed getting C0728762
Failed getting C0178638
Failed getting C0699129
Failed getting C0699129
Failed getting C0086787
Failed getting C0220892
Failed getting C0728762
Failed getting C0086787
Failed getting C0220892
Failed getting C0699034
Failed getting C0728762
Failed getting C0592168
Failed getting C0086787
Failed getting C

In [57]:
print(len(df))
errors = df[df.first_correct!=0]
ambiguous_error = errors[errors.pred==errors.mention]
len(ambiguous_error)/len(errors)

5302


0.19242902208201892

In [60]:
ambiguous_error[:50]

,mention,first_correct,pred_dist,correct,pred
6,difficulty,1,6,Difficult,difficulty
24,serax,-1,8,Product containing oxazepam (medicinal product),serax
39,serax,-1,8,Product containing oxazepam (medicinal product),serax
44,msir,-1,14,Morphine sulfate,msir
48,application,1,6,Application (procedure),application
60,hydration,3,10,Fluid management,hydration
61,application,1,6,Application (procedure),application
67,application,1,6,Application (procedure),application
198,injury,1,1,Injury,injury
212,more,1,4,More (qualifier value),more
